In [22]:

from datetime import date

from strava.data.strava_requests import (
    create_stream_df, 
    get_activity_stream_by_date, 
    activity_id_by_date,
)
from strava.data.cache import Cache
from strava.plotting.strava_stream_plots import (
    heartrate_with_altitude,
    velocity_with_altitude,
    heartrate_zones,
    all_streams,
    combine_plots_vertical,
)
from strava.plotting.strava_annual_plots import annual_plot
from generate_plots import get_clean_activities


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# General Setup

In [2]:
# Activities Cache
cache = Cache()

In [3]:
# Retrieve the list of activities
activities = get_clean_activities()

Existing access token is valid.
Beginning request of activities...
-> Received page 1 with 200 activities.
-> Received page 2 with 200 activities.
-> Received page 3 with 200 activities.
-> Received page 4 with 74 activities.
-> Received page 5 with 0 activities.
...retrieved total of 674 activities.


/Users/lucasnieuwenhout/Documents/Programming/Python/Projects/StravaPlotting/src/generate_plots.py:150: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.


In [4]:
activities

,id,name,distance,moving_time,elapsed_time,total_elevation_gain,elev_high,elev_low,sport_type,start_date_local,...,achievement_count,map.id,workout_type,average_speed,max_speed,average_heartrate,max_heartrate,map.summary_polyline,year,date_year_agnostic
1,15664199370,Afternoon Ride,21229.0,0 days 00:45:31,0 days 00:46:22,126.3,57.1,5.8,Ride,2025-09-01 13:24:50,...,3,a15664199370,NaN,27.9828,45.720,170.5,190.0,mdljH~rilV?a@AIEESHY?MBYPS@_@XKF]DoBAc@Oa@[OSY...,2025,1970-09-01 13:24:50
2,15653833274,Afternoon Ride,14598.6,0 days 00:32:28,0 days 00:33:01,105.6,57.1,5.8,Ride,2025-08-31 13:47:24,...,3,a15653833274,NaN,26.9784,47.520,169.0,194.0,kdljHnsilV?w@CEEAOHSAO@UN_@Fe@\MDO@iA@g@CQEQIc...,2025,1970-08-31 13:47:24
3,15630954354,Afternoon Ride,16280.8,0 days 00:35:54,0 days 00:36:07,110.8,57.1,5.8,Ride,2025-08-29 14:39:16,...,2,a15630954354,NaN,27.2088,43.992,173.0,193.0,mdljH|rilV?]AGCCK@EDS@OAEBQL]F_@T[J_@By@?e@EQE...,2025,1970-08-29 14:39:16
5,15586963677,Afternoon Ride,13019.6,0 days 00:30:13,0 days 00:30:54,100.2,57.1,5.8,Ride,2025-08-25 17:02:26,...,2,a15586963677,NaN,25.8516,45.648,176.2,193.0,odljH~silVAgAAGEAOJe@AGHOH_@F_@V[JQ@u@Au@Ia@M_...,2025,1970-08-25 17:02:26
6,15552409416,Afternoon Ride,11371.4,0 days 00:26:25,0 days 00:26:39,95.0,57.4,5.8,Ride,2025-08-22 17:28:45,...,1,a15552409416,NaN,25.8264,44.064,169.1,192.0,odljHbtilV?kAAEEAKHI@G@OCE@SPI@U?_@X[JO@wAAc@E...,2025,1970-08-22 17:28:45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,3425015993,Afternoon Ride,8964.6,0 days 00:26:43,0 days 00:26:43,39.0,693.8,654.7,Ride,2020-04-26 15:07:20,...,0,a3425015993,10.0,20.1312,40.320,165.0,185.0,{mjfIhhjtTt@fAf@|@HREZUpAe@pB{@vC]z@a@r@k@x@c@...,2020,1970-04-26 15:07:20
669,3425016238,Lunch Ride,17516.8,0 days 00:48:11,0 days 00:48:11,71.2,693.8,653.7,Ride,2020-04-25 11:48:43,...,0,a3425016238,10.0,21.8124,50.040,178.0,194.0,knjfIrhjtT`AxAb@v@G|@UvA[pAw@fCy@hBg@t@g@l@u@n...,2020,1970-04-25 11:48:43
670,3425017204,Afternoon Ride,14517.0,0 days 00:47:28,0 days 00:47:28,64.1,693.8,653.7,Ride,2020-04-23 13:02:13,...,0,a3425017204,10.0,18.3492,36.720,175.4,198.0,qnjfIlhjtTfAvAl@bAEp@OdA]hBc@lBW~@]~@Yj@g@t@_@...,2020,1970-04-23 13:02:13
671,3425016478,Afternoon Ride,17605.8,0 days 00:57:41,0 days 00:57:41,69.9,693.8,653.7,Ride,2020-04-22 14:48:16,...,0,a3425016478,10.0,18.3132,39.960,176.6,194.0,wmjfIdhjtTx@pAZv@F^?XAb@EZUt@[r@qBpCw@rA_@b@s@...,2020,1970-04-22 14:48:16


# Create Summary Plot

### Retrieve Stream

In [23]:
ride_date = date.today()
ride_date = date(2025, 9, 1)
stream = create_stream_df(
    get_activity_stream_by_date(activities=activities, 
                                date=ride_date, 
                                cache=cache), 
    activity_id_by_date(activities=activities, 
                        date=ride_date)
    )

### Create Plots

In [30]:
heartrate_plot = heartrate_with_altitude(stream=stream)
velocity_plot = velocity_with_altitude(stream=stream)
(heartrate_zone_plot, zone_plot) = heartrate_zones(stream=stream, heartrate_max=191)
summary = all_streams(stream, alpha=0.1)

### Save Plots

In [31]:
plot_height = 24
plot_width  = 48
transparent_background = False
heartrate_plot.save(filename="../plots/plot_components/heartrate_plot.png", format="png", height=plot_height, width=plot_width, limitsize=False, transparent=transparent_background)
velocity_plot.save(filename="../plots/plot_components/velocity_plot.png", format="png", height=plot_height, width=plot_width, limitsize=False, transparent=transparent_background)
heartrate_zone_plot.save(filename="../plots/plot_components/heartrate_zone_plot.png", format="png", height=plot_height, width=plot_width, limitsize=False, transparent=transparent_background)
zone_plot.save(filename="../plots/plot_components/zone_plot.png", format="png", height=4, width=48, limitsize=False, transparent=transparent_background)
summary.save(filename="../plots/plot_components/stream_summary.png", format="png", height=plot_height, width=plot_width, limitsize=False, transparent=transparent_background)

id = stream.iloc[0]["id"]
with open(f"../plots/summary_{ride_date.isoformat()}_{id}.png", "wb") as fp:
    combine_plots_vertical(["../plots/plot_components/velocity_plot.png", "../plots/plot_components/heartrate_plot.png", "../plots/plot_components/heartrate_zone_plot.png", "../plots/plot_components/zone_plot.png"]).save(fp=fp, format="png")

/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/layer.py:372: PlotnineWarning: geom_rect : Removed 1 rows containing missing values.
/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/geoms/geom_path.py:100: PlotnineWarning: geom_path: Removed 9 rows containing missing values.


# Create Annual Plots

In [33]:
subjects = ["heartrate", "speed", "distance", "elevation", "annual_time"]
plots = [annual_plot(activities=activities, subject=x) for x in subjects]

for i in range(len(subjects)):
    filename = f"../plots/annual/{subjects[i]}.png"
    plots[i].save(filename=filename, format="png", height=9, width=30, transparent=transparent_background, limitsize=False)

/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/layer.py:372: PlotnineWarning: geom_label : Removed 932 rows containing missing values.
/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/layer.py:372: PlotnineWarning: geom_label : Removed 932 rows containing missing values.
/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/scales/scale_manual.py:45: PlotnineWarning: The palette of scale_color_manual can return a maximum of 3 values. 6 were requested from it.


# Create Relevant Segments